In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import statistics

In [2]:
img_dir = "sample_images" # Enter Directory of all images 
data_path = os.path.join(img_dir,'*')
img_files = glob.glob(data_path)

In [3]:
#read pictures
shape_list = []
pad_list = []
for i in img_files:
    img = cv2.imread(i,cv2.IMREAD_GRAYSCALE)
    shape_list.append(img)
#add padd
for i in img_files:
    img = cv2.imread(i,cv2.IMREAD_GRAYSCALE)
    img3 = np.pad(img, pad_width=1, mode = 'constant', constant_values = 0)#adding 1 pixel pad
    img5 = np.pad(img, pad_width=2, mode = 'constant', constant_values = 0)#adding 2 pixel pad
    img7 = np.pad(img, pad_width=3, mode = 'constant', constant_values = 0)#adding 3 pixel pad
    img11 = np.pad(img, pad_width=5, mode = 'constant', constant_values = 0)#adding 5pixel pad
    
    #add padded pictures to a list
    pad_list.append(img3)
    pad_list.append(img5)
    pad_list.append(img7)
    pad_list.append(img11)

In [4]:
#creating kernels
#kernel bluring
kernel3 =  np.ones((3,3),np.float32)/9
kernel5 =  np.ones((5,5),np.float32)/25
kernel7 =  np.ones((7,7),np.float32)/49
kernel11 = np.ones((11,11),np.float32)/121

#kernel shifting up
shift_up_kernel = np.zeros((5,5),np.float32)
shift_up_kernel[4][2] = 1

#kernel shifting right
shift_right_kernel = np.zeros((7,7),np.float32)
shift_right_kernel[3][0] = 1

#kernel for sharpening
kernel_sharp = np.zeros((3,3),np.float32)
kernel_sharp[1][1] = 2
kernel_sharp = kernel_sharp - kernel3

#list for median
kernel_median = [0]*9

In [5]:
#creating new image
#listede 1 resmin 5 farkli cesidi var 5 e bol kac resim oldugunu bulmak icin
new_image = []
for i in range(len(pad_list)):
    x_k = pad_list[i].shape[0]#istedigin image
    y_k = pad_list[i].shape[1]
    new_image.append(np.zeros((x_k,y_k),np.uint8))#tupplellardan boyut al

In [6]:
def apply_kernel(i, j,kernel_size,img_num,isShift):
    summ=0
    prod=0
    kernel_index = 0
    s=0
    for k in range(kernel_size):#row of kernel
        for l in range(kernel_size):#column of kernel
            img = pad_list[img_num*4+kernel_num]#right padded image
            if isShift==1:#shift up
                kernel_index = shift_up_kernel[k][l]#finding item in kernel
            elif isShift==2:#shift right
                kernel_index = shift_right_kernel[k][l]#finding item in kernel
            elif isShift==3:#sharping
                kernel_index = kernel_sharp[k][l]#finding item in kernel
            elif isShift == 4:#salt and pepper filter
                kernel_median[s] = img[i+k][j+l]#adding item in kernel
                s+=1
            else:
                if kernel_size==3:
                    kernel_index = kernel3[k][l]#finding item in kernel
                elif kernel_size==5:
                    kernel_index = kernel5[k][l]#finding item in kernel
                elif kernel_size==7:
                    kernel_index = kernel7[k][l]#finding item in kernel
                elif kernel_size==11:
                    kernel_index = kernel11[k][l]#finding item in kernel
            img_index = img[i+k][j+l]#item of image respect to kernal
            prod = kernel_index * img_index
            summ = summ + prod
    if isShift == 4:#median of salt and pepper
        summ = statistics.median(kernel_median)
    return (summ)
#selin<3 

In [7]:
def filter2d(kernel_size,kernel_num,img_num,isShift):
    img_row = shape_list[img_num].shape[0]
    img_col = shape_list[img_num].shape[1]
    mid_kernel = (kernel_size-1)/2
    img = new_image[img_num*4+kernel_num]#right padded empty image
    a=0
    print(img_num," => ",end = "")# to track process
    for i in range(img_row):#row of image
        for j in range(img_col):#column of image
            summ=apply_kernel(i,j,kernel_size,img_num,isShift)#sum of items prouct by kernel
            img[i+int(mid_kernel)][j+int(mid_kernel)] = summ#placing sum to middle pixel of new image
        if i%100==0:# to track process
            print(img_row-i,end=" ")
    new_image[img_num*4+kernel_num] = img#replacing image to list
    print("")# to track process

In [9]:
#blur image 3x3
kernel_num=0
sub_img3 = []
for img_num in range(len(shape_list)):#traverse all images
    filter2d(3,kernel_num,img_num,0)#run blur filter QUESTION 1
    sub_img3.append(pad_list[img_num*4+kernel_num] - new_image[img_num*4+kernel_num])#sutract blur image from original. QUESTION 2
    
    #display image
    cv2.imshow('3x3', np.hstack((np.uint8(pad_list[img_num*4+kernel_num]),np.uint8(new_image[img_num*4+kernel_num]))))
    cv2.waitKey(0)
    cv2.destroyAllWindows() 
print('done')

0  => 512 412 312 212 112 12 
1  => 256 156 56 
2  => 400 300 200 100 
3  => 512 412 312 212 112 12 
4  => 240 140 40 
done


In [9]:
#blur image 5x5
kernel_num=1
sub_img5 = []
for img_num in range(len(shape_list)):#traverse all images
    filter2d(5,kernel_num,img_num,0)#run blur filter QUESTION 1
    sub_img5.append(pad_list[img_num*4+kernel_num] - new_image[img_num*4+kernel_num])#sutract blur image from original. QUESTION 2
    
    #display image
    cv2.imshow('5x5', np.hstack((np.uint8(pad_list[img_num*4+kernel_num]),np.uint8(new_image[img_num*4+kernel_num]))))
    cv2.waitKey(0)
    cv2.destroyAllWindows() 
print('done')

0  => 512 412 312 212 112 12 
1  => 256 156 56 
2  => 400 300 200 100 
3  => 512 412 312 212 112 12 
4  => 240 140 40 
done


In [ ]:
#blur image 7x7
kernel_num=2
sub_img7 = []
for img_num in range(len(shape_list)):#traverse all images
    filter2d(7,kernel_num,img_num,0)#run blur filter QUESTION 1
    sub_img7.append(pad_list[img_num*4+kernel_num] - new_image[img_num*4+kernel_num])#sutract blur image from original. QUESTION 2
    
    #display image
    cv2.imshow('7x7', np.hstack((np.uint8(pad_list[img_num*4+kernel_num]),np.uint8(new_image[img_num*4+kernel_num]))))
    cv2.waitKey(0)
    cv2.destroyAllWindows() 
print('done')

0  => 512 412 312 212 112 12 
1  => 256 156 56 
2  => 400 300 200 100 
3  => 512 412 312 212 112 12 
4  => 240 140 40 
done


In [ ]:
#blur image 11x11
kernel_num=3
sub_img11 = []
for img_num in range(len(shape_list)):#traverse all images
    filter2d(11,kernel_num,img_num,0)#run blur filter QUESTION 1
    sub_img11.append(pad_list[img_num*4+kernel_num] - new_image[img_num*4+kernel_num])#sutract blur image from original. QUESTION 2
    
    #display image
    cv2.imshow('11x11',np.hstack((np.uint8(pad_list[img_num*4+kernel_num]),np.uint8(new_image[img_num*4+kernel_num]))))
    cv2.waitKey(0)
    cv2.destroyAllWindows() 
print('done')

0  => 512 412 312 

In [10]:
for i in range(len(sub_img3)):#display question2
    cv2.imshow('subtract 3', np.uint8(sub_img3[i]))
    cv2.waitKey(0)
    cv2.destroyAllWindows() 

In [ ]:
for i in range(len(sub_img5)):#display question2
    cv2.imshow('subtract 5', np.uint8(sub_img5[i]))
    cv2.waitKey(0)
    cv2.destroyAllWindows() 

In [ ]:
for i in range(len(sub_img7)):#display question2
    cv2.imshow('subtract 7', np.uint8(sub_img7[i]))
    cv2.waitKey(0)
    cv2.destroyAllWindows() 

In [ ]:
for i in range(len(sub_img11)):#display question2
    cv2.imshow('subtract 11', np.uint8(sub_img11[i]))
    cv2.waitKey(0)
    cv2.destroyAllWindows() 

In [11]:
#shift up
kernel_num=1
shift_img = []
for img_num in range(len(shape_list)):
    filter2d(5,kernel_num,img_num,1)#run shift up filter QUESTION 3.1
    shift_img.append(new_image[img_num*4+kernel_num])#add to list
    
    #display shifted images
    cv2.imshow('shift up', np.hstack((np.uint8(pad_list[img_num*4+kernel_num]),(np.uint8(shift_img[img_num])))))
    cv2.waitKey(0)
    cv2.destroyAllWindows()

0  => 512 412 312 212 112 12 
1  => 256 156 56 
2  => 400 300 200 100 
3  => 512 412 312 212 112 12 
4  => 240 140 40 


In [ ]:
#shift right
kernel_num=2
shift_img = []
for img_num in range(len(shape_list)):
    filter2d(7,kernel_num,img_num,2)#run shift up filter QUESTION 3.2
    shift_img.append(new_image[img_num*4+kernel_num])#add to list
    
    #display shifted images
    cv2.imshow('shift right', np.hstack((np.uint8(shift_img[img_num]),(np.uint8(pad_list[img_num*4+kernel_num])))))
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [12]:
#sharping 3x3 filter img
kernel_num=0
sharp_img = []
for img_num in range(len(shape_list)):
    filter2d(3,kernel_num,img_num,3)#run sharpening filter Question 4
    sharp_img.append(new_image[img_num*4+kernel_num])#add to list
    
    #display shifted images
    cv2.imshow('sharpining', np.hstack((np.uint8(pad_list[img_num*4+kernel_num]),(np.uint8(sharp_img[img_num])))))
    cv2.waitKey(0)
    cv2.destroyAllWindows()

0  => 512 412 312 212 112 12 
1  => 256 156 56 
2  => 400 300 200 100 
3  => 512 412 312 212 112 12 
4  => 240 140 40 


In [ ]:
#salt and pepper
kernel_num = 0
img_num = 4
filter2d(3,kernel_num,img_num,4)#run salt and pepper filter

#display cleared image
cv2.imshow('salt and pepper', np.hstack((np.uint8(pad_list[img_num*4+kernel_num]),(np.uint8(new_image[img_num*4+kernel_num])))))
cv2.waitKey(0)
cv2.destroyAllWindows() 

4  => 240 140 40 
